# Whole brain data processing

In [1]:
import numpy as np
import pandas as pd
import os, sys
from glob import glob
from h5py import File
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2
from fish_proc.utils.memory import get_process_memory, clear_variables
cameraNoiseMat = '/groups/ahrens/ahrenslab/Ziqiang/gainMat/gainMat20180208'
dir_root = '/nrs/ahrens/Yu/SPIM/active_dataset/glia_neuron_imaging/20161109/fish2/20161109_2_1_6dpf_GFAP_GC_Huc_RG_GA_CL_fb_OL_f0_0GAIN_20161109_211950/raw'
import warnings
warnings.filterwarnings('ignore')
import dask.array as da
from utils import *

## Set up dask

In [2]:
import fish_proc.utils.dask_ as fdask
cluster, client = fdask.setup_workers(60)
client

Client Scheduler: tcp://10.36.111.11:36583 Dashboard: http://10.36.111.11:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


## Load file to dask

In [3]:
files = sorted(glob(dir_root+'/*.h5'))
# len_files = len(files)
# files = files[len_files//2-1000:len_files//2+1000]
chunks = File(files[0],'r')['default'].shape
data = da.stack([da.from_array(File(fn,'r')['default'], chunks=chunks) for fn in files])

## Pixel denoise

In [ ]:
from fish_proc.utils.getCameraInfo import getCameraInfo
cameraInfo = getCameraInfo(dir_root)
denoised_data = data.map_blocks(lambda v: pixelDenoiseImag(v, cameraInfo=cameraInfo))

## Registration

In [ ]:
med_win = len(denoised_data)
ref_img = denoised_data[med_win-100:med_win+100].mean(axis=0).compute()
np.save('motion_fix_', ref_img)
ref_img = np.load('motion_fix_.npy').max(0, keepdims=True)

In [ ]:
%%time
trans_affine = denoised_data[::10].map_blocks(lambda x: estimate_rigid2d(x, fixed=ref_img), dtype='object', drop_axis=(3), chunks=(1,4,4)).compute()
np.save('trans_affs', trans_affine)

In [ ]:
trans_affine_ = np.load('trans_affs.npy')
trans_affine_ = da.from_array(trans_affine_, chunks=(1,4,4))

In [ ]:
plt.plot(trans_affine_[:, 1, -1])
plt.plot(trans_affine_[:, 2, -1])
plt.show()

### Prepare data for video detrend

In [11]:
trans_data_ = da.map_blocks(apply_transform3d, denoised_data[::10], trans_affine_, chunks=(1, *denoised_data.shape[1:]), dtype='float32')

In [11]:
### these are identical
# trans_data = trans_data_.compute()
# for n in range(trans_data_.shape[1]):
#     da.to_npy_stack('data/%03d/'%(n), trans_data_[:, n, :, :], axis=0)
# trans_data_ = da.map_blocks(apply_transform3d, denoised_data[::10], trans_affine_, chunks=(1, *denoised_data.shape[1:]), dtype='float32')
# trans_data = trans_data_.compute()

In [23]:
%%time
trans_data_t = trans_data_.transpose((1, 2, 3, 0)).rechunk((1, 'auto', 'auto', -1))

CPU times: user 75 ms, sys: 4.96 ms, total: 80 ms
Wall time: 79.1 ms


In [26]:
%%time
import shutil
if os.path.exists('registered_data'):
    shutil.rmtree('registered_data')
trans_data_t.to_zarr('registered_data')

CPU times: user 1min 27s, sys: 6.17 s, total: 1min 33s
Wall time: 10min 52s


In [20]:
cluster.stop_all_jobs()
cluster.close()

Future exception was never retrieved
future: <Future finished exception=CommClosedError('in <closed TCP>: Stream is closed',)>
Traceback (most recent call last):
  File "/groups/ahrens/home/weiz/anaconda/envs/myenv/lib/python3.6/site-packages/distributed/comm/tcp.py", line 182, in read
    n_frames = yield stream.read_bytes(8)
  File "/groups/ahrens/home/weiz/anaconda/envs/myenv/lib/python3.6/site-packages/tornado/gen.py", line 1133, in run
    value = future.result()
tornado.iostream.StreamClosedError: Stream is closed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/groups/ahrens/home/weiz/anaconda/envs/myenv/lib/python3.6/site-packages/tornado/gen.py", line 1141, in run
    yielded = self.gen.throw(*exc_info)
  File "/groups/ahrens/home/weiz/anaconda/envs/myenv/lib/python3.6/site-packages/distributed/comm/tcp.py", line 202, in read
    convert_stream_closed_error(self, e)
  File "/groups/ahrens/home/weiz/anaconda/envs/

## Video Detrend

## Local PCA

## Demix